In [1]:
import pandas as pd
import numpy as np
import os

import numpy as np
import pandas as pd


In [2]:
dfm = pd.read_csv('https://www.immuno-compass.com/download/other/compass_gene_map.csv')
dfg = pd.read_json('https://www.immuno-compass.com/download/other/gene_tokens_long.json',
                   orient='index'
                   )[0]
cdict = pd.read_json('https://www.immuno-compass.com/download/other/cancer_code.json', 
                     orient='index')[0]

In [3]:
basef = 'https://raw.githubusercontent.com/ParkerICI/amadeus-trial-data/refs/heads/main/'
dfc = pd.read_csv(os.path.join(basef, 'clinical-data/AMADEUS_primarycohort_subject.csv'), index_col=0)
dftmb = pd.read_csv(os.path.join(basef, 'translational-data/AMADEUS_primarycohort_msi_tmb.csv'), index_col = 0)
dfc = dfc.join(dftmb[['msi', 'tmb']])
dfc['response_label'] = dfc['best.overall.response'].map({'Progressive Disease': 'NR',
 'Stable Disease': 'NR',
 'Partial Response': 'R',
 'Complete Response': 'R'})

#dfc['response_label'] = dfc['responder.flag.crpr'].map({'N': 'NR','Y': 'R'})

In [4]:
amadeus_to_tcga = {
    "Pancreatic carcinoma": "PAAD",               # Pancreatic adenocarcinoma
    "Sarcoma": "SARC",
    "Cervix cancer metastatic": "CESC",           # Cervical cancer
    "Breast cancer": "BRCA",
    "Non-small cell lung cancer": "LUAD",         # Mostly LUAD, general NSCLC
    "Ovarian cancer": "OV",
    "Thyroid cancer": "THCA",
    "Prostate cancer": "PRAD",
    "Head and neck cancer": "HNSC",
    "Gastric cancer": "STAD",                     # Stomach adenocarcinoma
    "Colorectal cancer": "COAD",                  # Prefer COAD, or READ
    "Mixed hepatocellular cholangiocarcinoma": "LIHC",  # Liver cancer
    "Uterine cancer": "UCEC",                     # Could include UCS, but UCEC more common
    "Peritoneal sarcoma": "SARC",
    "Neuroendocrine carcinoma": "OTHER",           # Not in TCGA, classify as "OTHER" or use SKCM-like model if skin derived
    "Hepatocellular carcinoma": "LIHC",
    "Pelvic neoplasm": "SARC",                    # Ambiguous, often mapped to soft tissue origin
    "Gastrooesophageal cancer": "STAD",           # Also overlaps with ESCA
    "Renal cancer": "KIRC",                       # Clear cell renal cell carcinoma (default)
    "Urethral cancer": "BLCA",                    # Urothelial origin, often grouped with BLCA
    "Penile cancer": "HNSC",                      # SCC, HPV-associated, closest to HNSC
    "Tumour of ampulla of Vater": "COAD",         # Intestinal-like subtype most common
    "Teratoma": "TGCT",                           # Testicular germ cell tumors
    "Neuroendocrine carcinoma of the skin": "SKCM"  # Most similar skin-origin carcinoma
}

In [5]:
dfc['tumor.type.tcga'] = dfc['tumor.type'].map(amadeus_to_tcga)

In [6]:
dft_pre = pd.read_csv(os.path.join(basef, 'translational-data/AMADEUS_primarycohort_rnaseq_baseline.csv'))
dft_post = pd.read_csv(os.path.join(basef, 'translational-data/AMADEUS_primarycohort_rnaseq_ontreatment.csv'))

/tmp/ipykernel_2588203/3447429017.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  dft_pre = pd.read_csv(os.path.join(basef, 'translational-data/AMADEUS_primarycohort_rnaseq_baseline.csv'))


In [7]:
dft_pre = dft_pre.groupby('subject.id').apply(lambda x:x.set_index('gene.name')['TPM.transcript.per.million'])
dft_post = dft_post.groupby('subject.id').apply(lambda x:x.set_index('gene.name')['TPM.transcript.per.million'])

In [20]:
dfc_pre.shape

(37, 30)

In [21]:
dfc_pre.arm.value_counts()

CD8 LOW     32
CD8 HIGH     5
Name: arm, dtype: int64

In [19]:
dft_post.shape

(25, 16706)

In [8]:
dfc_pre = dfc.loc[dft_pre.index]
dfc_post = dfc.loc[dft_post.index]

In [9]:
gs = dfg.loc[0:].tolist()

In [10]:
def prepare_dfx(dfx):
    diff = list(set(gs) - set(dfx.columns))
    dfamd = pd.DataFrame([], index = dfx.index, columns = diff).fillna(dfx.mean().min())

    return dfamd.join(dfx)[gs]

In [11]:
dfx_pre = prepare_dfx(dft_pre)
dfcx_pre = dfc_pre['tumor.type.tcga'].map(cdict).to_frame(name = 'cancer_code')
dfcx_pre = dfcx_pre.join(dfx_pre).fillna(0)

dfx_post = prepare_dfx(dft_post)
dfcx_post = dfc_post['tumor.type.tcga'].map(cdict).to_frame(name = 'cancer_code')
dfcx_post = dfcx_post.join(dfx_post).fillna(0)

dfcx_pre.shape, dfcx_post.shape

((37, 15673), (25, 15673))

In [26]:
dfc_post.arm.value_counts()

CD8 LOW     23
CD8 HIGH     2
Name: arm, dtype: int64

In [25]:
s.shape

(44,)

In [15]:
dfcx_pre.to_csv('./data/AMADEUS_Pre_TPM.tsv', sep = '\t')
dfcx_post.to_csv('./data/AMADEUS_Post_TPM.tsv', sep = '\t')

dfc_pre.to_csv('./data/AMADEUS_Pre_Clinical.tsv', sep = '\t')
dfc_post.to_csv('./data/AMADEUS_Post_Clinical.tsv', sep = '\t')

In [33]:
dfc_post.groupby(['arm', 'response_label']).size()

arm       response_label
CD8 HIGH  NR                 1
          R                  1
CD8 LOW   NR                14
          R                  6
dtype: int64

In [34]:
dfc_pre.groupby(['arm', 'response_label']).size()

arm       response_label
CD8 HIGH  NR                 4
          R                  1
CD8 LOW   NR                17
          R                  7
dtype: int64

In [40]:
pidx = dfc_pre.index.intersection(dfc_post.index)

In [44]:
dfc_pre.loc[pidx].groupby(['arm', 'response_label']).size()

arm       response_label
CD8 HIGH  NR                1
          R                 1
CD8 LOW   NR                9
          R                 4
dtype: int64